#### DATA MANIPULATION WITH PANDAS

<br>

## A/B Testing for ShoeFly.com

Our favorite online shoe store, ShoeFly.com is performing an A/B Test. They have two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. Help them analyze the data using aggregate measures.

<hr>

### Analyzing Ad Sources

In [1]:
import pandas as pd

ad_clicks = pd.read_csv('ad_clicks.csv')

Examine the first few rows of `ad_clicks`.

In [5]:
display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B


How many views (i.e., rows of the table) came from each `utm_source`?

In [4]:
views = ad_clicks.groupby('utm_source').user_id.count().reset_index()
views = views.rename(columns = {'user_id': 'count'})

display(views)

,utm_source,count
0,email,255
1,facebook,504
2,google,680
3,twitter,215


If the column `ad_click_timestamp` is not null, then someone actually clicked on the ad that was displayed.

In [6]:
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.notnull()

display(ad_clicks.head())

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False


We want to know the percent of people who clicked on ads from each `utm_source`.

In [7]:
clicks_by_source = ad_clicks.groupby(['utm_source', 'is_click']).user_id.count().reset_index()

display(clicks_by_source)

,utm_source,is_click,user_id
0,email,False,175
1,email,True,80
2,facebook,False,324
3,facebook,True,180
4,google,False,441
5,google,True,239
6,twitter,False,149
7,twitter,True,66


Now let’s pivot the data.

In [8]:
clicks_pivot = clicks_by_source.pivot(
  columns = 'is_click',
  index = 'utm_source',
  values = 'user_id'
).reset_index()

display(clicks_pivot)

is_click,utm_source,False,True
0,email,175,80
1,facebook,324,180
2,google,441,239
3,twitter,149,66


Create a new column in `clicks_pivot` called `percent_clicked` which is equal to the percent of users who clicked on the ad from each `utm_source`.

In [9]:
clicks_pivot['percent_clicked'] = clicks_pivot[True] / (clicks_pivot[True] + clicks_pivot[False])

display(clicks_pivot)

is_click,utm_source,False,True,percent_clicked
0,email,175,80,0.313725
1,facebook,324,180,0.357143
2,google,441,239,0.351471
3,twitter,149,66,0.306977


<hr>

### Analyzing an A/B Test

The column `experimental_group` tells us whether the user was shown Ad A or Ad B. Were approximately the same number of people shown both ads?

In [10]:
experiment = ad_clicks.groupby('experimental_group').user_id.count().reset_index()

display(experiment)

,experimental_group,user_id
0,A,827
1,B,827


Check to see if a greater percentage of users clicked on Ad A or Ad B.

In [11]:
a_or_b = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()

display(a_or_b)

,experimental_group,is_click,user_id
0,A,False,517
1,A,True,310
2,B,False,572
3,B,True,255


In [12]:
a_or_b_pivot = a_or_b.pivot(
  columns = 'is_click',
  index = 'experimental_group',
  values = 'user_id'
).reset_index()

display(a_or_b_pivot)

is_click,experimental_group,False,True
0,A,517,310
1,B,572,255


The Product Manager for the A/B test thinks that the clicks might have changed by day of the week. Start by creating two DataFrames: `a_clicks` and `b_clicks`.

In [13]:
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A']

display(a_clicks)

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A,True
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A,False
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A,False
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A,False
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A,False
...,...,...,...,...,...,...
1643,fceb13ea-fd8c-446a-a61f-f977d404330a,twitter,6 - Saturday,NaN,A,False
1646,fd7d06ea-38b5-4ed9-acc9-777047db8c56,google,4 - Thursday,NaN,A,False
1647,fe570a20-448f-40ed-930b-8482b8a7c231,facebook,1 - Monday,20:7,A,True
1649,fe8b5236-78f6-4192-9da6-a76bba67cfe6,twitter,7 - Sunday,NaN,A,False


In [14]:
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B']

display(b_clicks)

,user_id,utm_source,day,ad_click_timestamp,experimental_group,is_click
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B,False
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B,False
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B,False
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B,True
10,01adb2e7-f711-4ae4-a7c6-29f48457eea1,google,3 - Wednesday,NaN,B,False
...,...,...,...,...,...,...
1645,fd2a5852-f0ef-4162-84a6-107a42dc46b5,twitter,3 - Wednesday,NaN,B,False
1648,fe6cfa5a-cc63-4770-8d56-c13ac8cf5bef,google,3 - Wednesday,15:6,B,True
1650,fed3db6d-8c92-40e3-a4fb-1fb9d7337eb1,facebook,5 - Friday,NaN,B,False
1651,ff3a22ff-521c-478c-87ca-7dc7b8f34372,twitter,3 - Wednesday,NaN,B,False


For each group, calculate the percent of users who clicked on the ad by `day`.

In [15]:
a_clicks_day = a_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()

display(a_clicks_day)

,is_click,day,user_id
0,False,1 - Monday,70
1,False,2 - Tuesday,76
2,False,3 - Wednesday,86
3,False,4 - Thursday,69
4,False,5 - Friday,77
5,False,6 - Saturday,73
6,False,7 - Sunday,66
7,True,1 - Monday,43
8,True,2 - Tuesday,43
9,True,3 - Wednesday,38


In [16]:
a_clicks_day_pivot = a_clicks_day.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()

a_clicks_day_pivot['percent_clicked'] = a_clicks_day_pivot[True] / (a_clicks_day_pivot[True] + a_clicks_day_pivot[False])

display(a_clicks_day_pivot)

is_click,day,False,True,percent_clicked
0,1 - Monday,70,43,0.380531
1,2 - Tuesday,76,43,0.361345
2,3 - Wednesday,86,38,0.306452
3,4 - Thursday,69,47,0.405172
4,5 - Friday,77,51,0.398438
5,6 - Saturday,73,45,0.381356
6,7 - Sunday,66,43,0.394495


In [17]:
b_clicks_day = b_clicks.groupby(['is_click', 'day']).user_id.count().reset_index()

display(b_clicks_day)

,is_click,day,user_id
0,False,1 - Monday,81
1,False,2 - Tuesday,74
2,False,3 - Wednesday,89
3,False,4 - Thursday,87
4,False,5 - Friday,90
5,False,6 - Saturday,76
6,False,7 - Sunday,75
7,True,1 - Monday,32
8,True,2 - Tuesday,45
9,True,3 - Wednesday,35


In [18]:
b_clicks_day_pivot = b_clicks_day.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
).reset_index()

b_clicks_day_pivot['percent_clicked'] = b_clicks_day_pivot[True] / (b_clicks_day_pivot[True] + b_clicks_day_pivot[False])

display(b_clicks_day_pivot)

is_click,day,False,True,percent_clicked
0,1 - Monday,81,32,0.283186
1,2 - Tuesday,74,45,0.378151
2,3 - Wednesday,89,35,0.282258
3,4 - Thursday,87,29,0.250000
4,5 - Friday,90,38,0.296875
5,6 - Saturday,76,42,0.355932
6,7 - Sunday,75,34,0.311927


<hr>

### Summary

It would be better to use Ad A since it is more likely that the ad will be clicked on over Ad B.